In [37]:
import luigi
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '/home/fbuonerba/codes/')
from mp_functions import upload_log_return, upload_factor_loadings
from coinapi_v1 import CoinAPIv1
import datetime
from datetime import datetime, timedelta
import time
import calendar
import json
import urllib.request
import multiprocessing as mp
import sklearn
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [9]:
#####Returns: January to April#####

with open('/home/fbuonerba/codes/meta_data/top_coins.txt') as ff:
    coins=json.load(ff)
coins=list(coins)
####LIZA is outlier for 2 of the risk factors...####
coins.remove('LIZA')
quotes=['USD','BTC']
#sysm=[str(coin)+'_'+str(quote) for coin in coins for quote in quotes]
t=1515283200
matrix=[]
#matrix of daily log_returns (time, base_quote)
while t<=1522540801:
    ret_t=[]
    for base in coins:
        for quote in quotes:
            returns=upload_log_return(t, base, quote, 86400)
            if np.isnan(returns)==True:
                returns=0
            ret_t.append(returns)
            #print(base,quote)
    matrix.append(ret_t)
    t+=86400
R=np.array(matrix)

In [38]:
#####Returns: April 2nd to May 2nd#####

with open('/home/fbuonerba/codes/meta_data/top_coins.txt') as ff:
    coins=json.load(ff)
coins=list(coins)
####LIZA is outlier for 2 of the risk factors...####
coins.remove('LIZA')
quotes=['USD','BTC']
#sysm=[str(coin)+'_'+str(quote) for coin in coins for quote in quotes]
t=1522627200
matrix=[]
#matrix of daily log_returns (time, base_quote)
while t<=1525219201:
    ret_t=[]
    for base in coins:
        for quote in quotes:
            returns=upload_log_return(t, base, quote, 86400)
            if np.isnan(returns)==True:
                returns=0
            ret_t.append(returns)
            #print(base,quote)
    matrix.append(ret_t)
    t+=86400
R=np.array(matrix)

In [39]:
#####Factors: January to April#####

factors=[]
factor_names1=['variance_', 'strength_', 'high_low_']
factor_names2=['turnover_', 'log_mkcap_']
factor_names3=['coin_ratio_']
path='/home/fbuonerba/factor_loadings/'
finpath1='_'+str(1515283200)+'_'+str(1522540800)+'_'+str(604800)+'.txt'
finpath2='_'+str(1515283200)+'_'+str(1522540800)+'.txt'
finpath3='.txt'
for base in coins:
    for quote in quotes:
        sym=str(base)+'_'+str(quote)
        sym_row=[]
        for name in factor_names1:
            with open(path+name+sym+finpath1) as file:
                x=json.load(file)
            sym_row.append(x)
        for name in factor_names2:
            with open(path+name+sym+finpath2) as file:
                x=json.load(file)
            sym_row.append(x)
        for name in factor_names3:
            #symll=sym.split('_')
            with open(path+name+base+finpath3) as file:
                x=json.load(file)
            sym_row.append(x)
        #sym_row.append(1)
        factors.append(sym_row)
factors=np.array(factors)  
#normalization using z-score along coin axis:
factors=(factors-np.mean(factors,axis=0))/np.var(factors,axis=0)**.5

In [95]:
print('This model is tested on daily returns, taken from April 2nd to May 2nd.')
print('Factor loadings have been computed for April 1st.')
print('Hence there are: 88 coin pairs; 31 days of returns; 6 factors')
print('The dimensionality of various variables are listed below:')
print('factors size =', factors.shape)
print('returns size =', R.shape)
reg = linear_model.LinearRegression()
reg.fit(factors, R.T)
beta_=reg.coef_
inter_=reg.intercept_
R_hat=np.dot(factors,beta_.T)+inter_
print('beta size =', beta_.shape)
print('beta =', beta_)
print('intercept =', inter_)
#average daily returns
mean=np.mean(R,axis=1)
#tts,rss,R^2 and F-score.
tss=np.linalg.norm(R.T-mean)**2
rss=np.linalg.norm(R.T-R_hat)**2
r_squared=1-rss/tss
print('r_squared =', r_squared)
F_score_num=(tss-rss)/(factors.shape[1])
F_score_denom=rss/(factors.shape[0]-factors.shape[1])
F_score=F_score_num/F_score_denom
print('F_score =', F_score)
#variance of residuals: var(res)=Expected(res^T.res)/n-p
var_residuals=F_score_denom
#var(beta)=(factors^T.factors)^{-1}.var(error)
#also, var(beta)=np.cov(beta_.T)
var_beta=np.linalg.inv(np.dot(factors.T,factors))*var_residuals
var_beta1=np.cov(beta_.T)
#t_stat(beta_k,t)= beta_k,t/std(beta_k)
t_stats=beta_/np.sqrt(var_beta1.diagonal())
print('t_stats =', t_stats)


This model is tested on daily returns, taken from April 2nd to May 2nd.
Factor loadings have been computed for April 1st.
Hence there are: 88 coin pairs; 31 days of returns; 6 factors
The dimensionality of various variables are listed below:
factors size = (88, 6)
returns size = (31, 88)
beta size = (31, 6)
beta = [[ 5.67285347e-03 -8.90913003e-03 -7.72425396e-03  8.47673231e-03
   1.44291001e-02 -7.55970413e-03]
 [-1.27275178e-03 -8.89261713e-03  2.77720774e-02 -5.39301128e-03
   1.22477222e-03  1.99010636e-02]
 [-6.88861835e-03  2.83344652e-02  1.08384064e-02  4.33386783e-03
  -2.92683280e-02 -7.52395963e-04]
 [ 1.59469516e-02 -4.04147801e-03 -1.74845779e-02  8.68295729e-03
   1.19082854e-02 -1.05542688e-02]
 [-6.46167952e-03  1.52118871e-02  1.27884637e-02  7.12477945e-04
  -1.03773890e-02  1.82527621e-04]
 [ 5.70669419e-03 -1.34987286e-02 -1.27463007e-02  4.45607566e-04
   1.11487654e-02  1.92021079e-03]
 [ 2.43853581e-03 -9.02483175e-03 -1.76349789e-03 -1.19889535e-03
   4.5386587

In [ ]:
#organize a file#
diction={}
diction['mean_squared_error']=mean_squared_error(R.T, R_hat)
diction['r_squared']=reg.score(factors,R.T)
diction['normalized_factor_loadings']=factors
diction['returns']=R.T
diction['factor_returns']=beta_

diction

In [ ]:
#####February to May#####

from mp_functions import upload_log_return
with open('/home/fbuonerba/codes/meta_data/top_coins.txt') as ff:
    coins=json.load(ff)
coins=list(coins)
####LIZA is outlier for 2 of the risk factors...####
coins.remove('LIZA')
quotes=['USD','BTC']
#sysm=[str(coin)+'_'+str(quote) for coin in coins for quote in quotes]
t=1517702400
matrix=[]
#matrix of daily log_returns (time, base_quote)
while t<=1525564801:
    ret_t=[]
    for base in coins:
        for quote in quotes:
            returns=upload_log_return(t, base, quote, 86400)
            if np.isnan(returns)==True:
                returns=0
            ret_t.append(returns)
            #print(base,quote)
    matrix.append(ret_t)
    t+=86400
R=np.array(matrix)

factors=[]
factor_names1=['variance_', 'strength_', 'high_low_']
factor_names2=['turnover_', 'log_mkcap_']
factor_names3=['coin_ratio_']
path='/home/fbuonerba/codes/factor_loadings/'
finpath1='_'+str(1517702400)+'_'+str(1525564800)+'_'+str(604800)+'.txt'
finpath2='_'+str(1517702400)+'_'+str(1525564800)+'.txt'
finpath3='.txt'
for base in coins:
    for quote in quotes:
        sym=str(base)+'_'+str(quote)
        sym_row=[]
        for name in factor_names1:
            with open(path+name+sym+finpath1) as file:
                x=json.load(file)
            sym_row.append(x)
        for name in factor_names2:
            with open(path+name+sym+finpath2) as file:
                x=json.load(file)
            sym_row.append(x)
        for name in factor_names3:
            #symll=sym.split('_')
            with open(path+name+base+finpath3) as file:
                x=json.load(file)
            sym_row.append(x)
        #sym_row.append(1)
        factors.append(sym_row)
factors=np.array(factors)  
#normalization using z-score along coin axis:
factors=(factors-np.mean(factors,axis=0))/np.var(factors,axis=0)**.5
#X_with_ones = np.hstack((np.ones((factors.shape[0], 1)), factors))

In [ ]:
reg = linear_model.LinearRegression()
reg.fit(factors, R.T)
#reg.fit pre-processes factors - centers along axis=0
#adding a colums of ones to factors makes no difference, there will be intercept anyway.
beta_=reg.coef_
inter_=reg.intercept_

R_hat1=reg.predict(factors)
R_hat=np.dot(factors,beta_.T)+inter_
print('r2 using daily mean =',reg.score(factors,R.T), r2_score(R.T,R_hat1,multioutput='variance_weighted'))
print('r2 using total mean =',r2_score(R.T,R_hat),)
#reg.score computes the mean along axis=0
#r2_score computes the mean over full matrix


In [ ]:
#factors must have extra column of ones for this to work.
factors=np.hstack((np.ones((factors.shape[0], 1)), factors))
beta=np.linalg.lstsq(factors, R.T,rcond=None)
gamma=beta[0]
R_hat_np=np.dot(factors,gamma)
residuals=np.sum( (R.T-R_hat_np)**2)
#average returns over days, for each coin
mean=np.mean(R, axis=0)
sum_squares=np.sum((R-mean.T)**2)
r2_coin=1-residuals/sum_squares
#average returns over coins, for each day.
mean_bs=np.mean(R, axis=1)
sum_squares_bs=np.sum((R.T-mean_bs)**2)
r2_daily=1-residuals/sum_squares_bs
print('r2 using coin mean =',r2_coin)
print('r2 using daily mean =',r2_daily)
#just average over days and coins
print('r2 using total mean =',1-residuals/np.linalg.norm(R.T-np.mean(R))**2)

In [ ]:
######################################################

In [ ]:
#regression against each one single time, BTC/USD
factor_test=factors
r=[]
for i in range(85):
    R_test=R[i]
    beta_test=np.linalg.lstsq(factor_test,R_test.T, rcond=None)
    S=np.dot(factors,beta_test[0])
    res_test=np.linalg.norm(R_test-S)**2
    r.append(1-res_test/np.linalg.norm(R_test)**2)
r=np.array(r)
print(np.max(r), np.min(r))

In [ ]:
reg=linear_model.LinearRegression()
for i in range(6):
    fac=factors.T[i].reshape(-1,1)
    ret=R[80].T
    #print(fac.shape)
    reg.fit(fac,ret)
    temp_beta=reg.coef_[0]
    temp_inter=reg.intercept_
    print(temp_beta, temp_inter)
    print(reg.score(fac,ret))
    import matplotlib.pyplot as plt
    plt.plot(fac, ret,'o')
    plt.plot(fac, temp_beta*fac + temp_inter, 'r', label='Fitted line')
    plt.show()

In [ ]:
for t in range(88):
    print(coins[int(t/2)],quotes[t%2])
    plt.plot(range(len(R)), R.T[t])
    plt.show()

In [7]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [33]:
factors=sm.add_constant(factors)
X=factors
y=R.T
results = sm.OLS(y,X).fit()
results

AttributeError: 'OLSResults' object has no attribute 'tstat'